In [18]:


import os
import cv2
import numpy as np
import shutil
from PIL import Image



In [41]:
os.chdir(
    "F:/Users/elect_09l/github/ship_detection/YOLO_MODEL"
)  # Set the base directory of the script and where other files will be created

dir_base = r"F:/Users/elect_09l/github/ship_detection"
dir_yolo = dir_base + "/YOLO_MODEL/"

dir_folder_list = [
    dir_yolo + "analysed_imgs"
]

folder_list = ["analysed_imgs"]

if BOOL_DELETE_DIR:
    delete_dir()
if BOOL_MAKE_DIR:
    make_dir()


# Load YOLO Model
net = cv2.dnn.readNet(
    "D:/py/project/YOLO/" + "yolov3.weights", dir_yolo + "yolov3.cfg"
)  # load weights dir and configurations dir

classes = []
with open(dir_yolo + "coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

line_pts = (
    []
)  # List of all the points of detection by the model, tuples are added through each iteration of the loop below

first_iter = True

# Settings for the "Start Point" (shows the beginning of the path in which the ship moves in)
start_pt_radius = 15
start_pt_colour = (23, 144, 255)
start_pt_centre = (0, 0)

In [43]:
# read input image
img = "./input/download.jpg"
print(img)
image = cv2.imread(img)

Width = image.shape[1]
Height = image.shape[0]
scale = 0.00392

img_size = (Width, Height)

# read class names from text file
# generate different colors for different classes 
COLORS = np.random.uniform(0, 255, size=(len(classes), 3))

# create input blob 
blob = cv2.dnn.blobFromImage(image, scale, (416,416), (0,0,0), True, crop=False)

# set input blob for the network
net.setInput(blob)

./input/download.jpg


327

In [44]:
# function to get the output layer names 
# in the architecture
def get_output_layers(net):
    
    layer_names = net.getLayerNames()
    
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

    return output_layers

# function to draw bounding box on the detected object with class name
def draw_bounding_box(img, class_id, confidence, x, y, x_plus_w, y_plus_h):

    label = str(classes[class_id])

    color = COLORS[class_id]

    cv2.rectangle(img, (x,y), (x_plus_w,y_plus_h), color, 2)

    cv2.putText(img, label, (x-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

In [45]:
# run inference through the network
# and gather predictions from output layers
outs = net.forward(get_output_layers(net))

# initialization
class_ids = []
confidences = []
boxes = []
conf_threshold = 0.5
nms_threshold = 0.4

# for each detetion from each output layer 
# get the confidence, class id, bounding box params
# and ignore weak detections (confidence < 0.5)
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            center_x = int(detection[0] * Width)
            center_y = int(detection[1] * Height)
            w = int(detection[2] * Width)
            h = int(detection[3] * Height)
            x = center_x - w / 2
            y = center_y - h / 2
            class_ids.append(class_id)
            confidences.append(float(confidence))
            boxes.append([x, y, w, h])

In [46]:
# apply non-max suppression
indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)

# go through the detections remaining
# after nms and draw bounding box
for i in indices:
    i = i[0]
    box = boxes[i]
    x = box[0]
    y = box[1]
    w = box[2]
    h = box[3]
    
    draw_bounding_box(image, class_ids[i], confidences[i], round(x), round(y), round(x+w), round(y+h))

FILENAME = classes[class_ids[i]]
    
# display output image    
cv2.imshow("object detection", image)

# wait until any key is pressed
cv2.waitKey()
    
 # save output image to disk
cv2.imwrite(f"output/{FILENAME}.jpg", image)

# release resources
cv2.destroyAllWindows()

In [38]:
print(class_ids)

[8, 8, 8, 8, 8, 8]


In [39]:
str(classes[class_ids[i]])

'boat'

In [ ]:
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

In [58]:
heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name)

AttributeError: 'cv2.dnn_Net' object has no attribute 'layers'